<a href="https://colab.research.google.com/github/KEHSANN/hosh_masnoi/blob/main/miniproject/3/m3_q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Q4 part 1**

In [82]:
!pip install --upgrade --no-cache-dir gdown
!gdown 1NGup5nez1AtiJqvkDAjU3ftyegFAgVeu

Downloading...
From: https://drive.google.com/uc?id=1NGup5nez1AtiJqvkDAjU3ftyegFAgVeu
To: /content/covid.csv
100% 258/258 [00:00<00:00, 1.05MB/s]


In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
data = pd.read_csv('/content/covid.csv')
data

,Fever,Cough,Breathing issues,Infected
0,No,No,No,No
1,Yes,Yes,Yes,Yes
2,Yes,Yes,No,No
3,Yes,No,Yes,Yes
4,Yes,Yes,Yes,Yes
5,No,Yes,No,No
6,Yes,No,Yes,Yes
7,Yes,No,Yes,Yes
8,No,Yes,Yes,Yes
9,Yes,Yes,No,Yes


In [7]:
labels = data['Infected']
p = labels.value_counts() / len(labels)
print(-sum(p * np.log2(p)))
p


0.9852281360342515


Yes    0.571429
No     0.428571
Name: Infected, dtype: float64

In [8]:
def entropy(labels):
    p = labels.value_counts() / len(labels)
    return -sum(p * np.log2(p))

data['Infected'].value_counts()

Yes    8
No     6
Name: Infected, dtype: int64

In [17]:
entropy(data['Infected'])# آنتروپی پرنت برابراست با

0.9852281360342515

In [21]:
 entropy_child = 0
 for value in data['Cough'].unique():
        subset = data[data['Cough'] == value]
        print(subset)
        wi = len(subset) / len(data)
        entropy_child += wi * entropy(subset['Infected'])
entropy_child

  Fever Cough Breathing issues Infected
0    No    No               No       No
3   Yes    No              Yes      Yes
6   Yes    No              Yes      Yes
7   Yes    No              Yes      Yes
   Fever Cough Breathing issues Infected
1    Yes   Yes              Yes      Yes
2    Yes   Yes               No       No
4    Yes   Yes              Yes      Yes
5     No   Yes               No       No
8     No   Yes              Yes      Yes
9    Yes   Yes               No      Yes
10    No   Yes               No       No
11    No   Yes              Yes      Yes
12    No   Yes              Yes       No
13   Yes   Yes               No       No


0.9460794641311808

# **Information Gain**

In [22]:
def information_gain(data, feature, target):
    # Entropy of parent
    entropy_parent = entropy(data[target])

    # Entropy of child
    entropy_child = 0
    for value in data[feature].unique():
        subset = data[data[feature] == value]
        #display(subset)
        wi = len(subset) / len(data)
        entropy_child += wi * entropy(subset[target])

    return entropy_parent - entropy_child

arg=[information_gain(data, feature, 'Infected') for feature in data.iloc[:, :-1].columns]#iloc[:, :-1] برای اینه که خود تارگت رو حساب نکنه

In [ ]:
np.argmax(arg)

2

# **2 is Breathing issues so Breathing issues be selected**

# **creating Node:**

In [97]:



class Node:
    def __init__(self, feature=None, label=None, node_type=None):
        self.feature = feature        # feature used for splitting
        self.label = label            # label if leaf node
        self.node_type = node_type    # 'LEAF' or 'SPLIT'
        self.children = {}
        self.n = 1



    def __repr__(self):

        return f'{node_type}Node({self.feature}={self.label})' if self.node_type == 'LEAF' \
            else f'{node_type}Node({self.feature})={{children}}'


    def __repr__(self): # کمک میکنه که خروجی آدرس نباشه و یک چیز قابل فهم باشه خروجی
        node_type = 'Leaf' if self.node_type == 'LEAF' else 'Decision'
        if self.feature is not None:
            return f'DecisionNode(feature="{self.feature}", children={self.children})'
            #وقتی دیسیژن نود داشته باشیم حتما چیلدرن نود هم داریم
        else:
            return f'LeafNode(label="{self.label}")'

Node(label='Cough')
# Node(label='yes')
#زمانی که دیسیژن نود داشته باشیم حالت اون فیچر میشه و زمانی که لیف نود داشته باشیم حالت نان میمونه
#پس ما از بیرون به کلاس نود خودمون فقط یا فیچر میدیم یا لیبل میدیم

LeafNode(label="Cough")

# **Decision Tree**

In [35]:
def make_treee(data, target):



    features = data.drop(target, axis=1).columns# ازش خود تارگت رو حذف کنیم
    print(features)


    for xy in features:
      print(xy)

make_treee(data, 'Infected')

Index(['Fever', 'Cough', 'Breathing issues'], dtype='object')
Fever
Cough
Breathing issues


In [ ]:
  for feature in features:

In [45]:
def make_treeee(data, target):


    features = data.drop(target, axis=1).columns# ازش خود تارگت رو حذف کنیم
    gain=[]
    for feature in features:
      IG=information_gain(data, feature, target)
      gain.append(IG)




    # calculate information gain
    gains = [information_gain(data, feature, target) for feature in features]
    return gain
make_treeee(data, 'Infected')

[0.12808527889139443, 0.0391486719030707, 0.39603884492804464]

In [98]:
def make_tree(data, target):
    # leaf node?
    # check if leaf node
    if len(data[target].unique()) == 1:
        return Node(label=data[target].iloc[0], node_type='LEAF')
    # check if no more features to split
    features = data.drop(target, axis=1).columns
    if len(features) == 0:
        return Node(label=data[target].mode()[0], node_type='LEAF')


    # calculate information gain
    gains = [information_gain(data, feature, target) for feature in features]#در کدی که بالاتر نوشتم معادل این کد نوشته شده
    # greedy search to find best feature
    max_gain_idx = np.argmax(gains)#میخواهیم اندیسشو بهمون بده تا بریم در دیتا با اون کار کنیم
    best_feature = features[max_gain_idx]

    # make a node
    node = Node(feature=best_feature, node_type='SPLIT')





    # loop over the best feature
    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]#در ساب ست دیگه به خود اون ستون نیازی نداریم
        display(subset)
        subset = data[data[best_feature] == value].drop(best_feature, axis=1)
        child_node = make_tree(subset, target)
        node.children[value] = child_node
        node.n += child_node.n


    return node

In [99]:
tree = make_tree(data, 'Infected')
tree

,Fever,Cough,Breathing issues,Infected
0,No,No,No,No
2,Yes,Yes,No,No
5,No,Yes,No,No
9,Yes,Yes,No,Yes
10,No,Yes,No,No
13,Yes,Yes,No,No


,Fever,Cough,Infected
0,No,No,No
5,No,Yes,No
10,No,Yes,No


,Fever,Cough,Infected
2,Yes,Yes,No
9,Yes,Yes,Yes
13,Yes,Yes,No


,Cough,Infected
2,Yes,No
9,Yes,Yes
13,Yes,No


,Fever,Cough,Breathing issues,Infected
1,Yes,Yes,Yes,Yes
3,Yes,No,Yes,Yes
4,Yes,Yes,Yes,Yes
6,Yes,No,Yes,Yes
7,Yes,No,Yes,Yes
8,No,Yes,Yes,Yes
11,No,Yes,Yes,Yes
12,No,Yes,Yes,No


,Fever,Cough,Infected
1,Yes,Yes,Yes
3,Yes,No,Yes
4,Yes,Yes,Yes
6,Yes,No,Yes
7,Yes,No,Yes


,Fever,Cough,Infected
8,No,Yes,Yes
11,No,Yes,Yes
12,No,Yes,No


,Cough,Infected
8,Yes,Yes
11,Yes,Yes
12,Yes,No


DecisionNode(feature="Breathing issues", children={'No': DecisionNode(feature="Fever", children={'No': LeafNode(label="No"), 'Yes': DecisionNode(feature="Cough", children={'Yes': LeafNode(label="No")})}), 'Yes': DecisionNode(feature="Fever", children={'Yes': LeafNode(label="Yes"), 'No': DecisionNode(feature="Cough", children={'Yes': LeafNode(label="Yes")})})})

In [69]:
tree.feature

'Breathing issues'

**اطلاعات بیش تر**

# Q4 Part 2